In [18]:
# Import utility.py and db_utility
import scripts.utils as ut
from scripts.db_utils import get_db_engine, open_connection, close_connection, get_table_count
from sqlalchemy import text
from sqlalchemy.exc import SQLAlchemyError

# csv columns cleaned file read
df = ut.read_world_colCleanDataset()

In [16]:
# Columns for the first table => regcit
df_cities = df[['city', 'country']].drop_duplicates().reset_index(drop=True)

In [22]:
# DB Connection 
engine = get_db_engine()
conn = open_connection(engine)

# Counts
inserted = 0
skipped = 0

# Insert on regcit table
for _, row in df_cities.iterrows():
    try:
        conn.execute(
            text("""
                INSERT INTO regcit (namecit, country)
                SELECT :name, :country
                FROM DUAL
                WHERE NOT EXISTS (
                    SELECT 1 FROM regcit
                    WHERE namecit = :name AND country = :country
                )
            """),
            {"name": row['city'], "country": row['country']}
        )
        inserted += 1
        
    except SQLAlchemyError as e:
        print(f"❌ Error inserting {row['city']}, {row['country']}: {e}")
        skipped += 1

conn.commit()

print(f"✅ regcit population completed: {inserted} inserted, {skipped} skipped.")

result = conn.execute(text("""SELECT * FROM regcit LIMIT 10;"""))

for row in result:
    print(row)

# Close db connection 
close_connection(conn)

🔍 Loading .env from: C:\Users\Matteo\progetti\real-estate\.env
🔧 Variables loaded: user=user, host=localhost, db=livingcost
✅ regcit population completed: 4874 inserted, 0 skipped.
(1, 'Delhi', 'India')
(2, 'Shanghai', 'China')
(3, 'Jakarta', 'Indonesia')
(4, 'Manila', 'Philippines')
(5, 'Seoul', 'South Korea')
(6, 'Bangkok', 'Thailand')
(7, 'Kolkata', 'India')
(8, 'Guangzhou', 'China')
(9, 'Mumbai', 'India')
(10, 'Beijing', 'China')
✅ DB connection closed


In [23]:
# Total num of rows inserted

engine = get_db_engine()
conn = open_connection(engine)

total_rows = get_table_count(conn, "regcit")
print(f"✅ Total rows in regcit: {total_rows}")

close_connection(conn)

🔍 Loading .env from: C:\Users\Matteo\progetti\real-estate\.env
🔧 Variables loaded: user=user, host=localhost, db=livingcost
✅ Total rows in regcit: 4874
✅ DB connection closed
